<a href="https://colab.research.google.com/github/google-research/tapas/blob/master/notebooks/tabfact_predictions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##### Copyright 2020 The Google AI Language Team Authors

Licensed under the Apache License, Version 2.0 (the "License");

In [0]:
# Copyright 2019 The Google AI Language Team Authors.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

Running a Tapas fine-tuned checkpoint
---
This notebook shows how to load and make predictions with TAPAS model, which was introduced in the paper: [TAPAS: Weakly Supervised Table Parsing via Pre-training](https://arxiv.org/abs/2004.02349)

# Clone and install the repository


First, let's install the code.

In [0]:
! pip install tapas-table-parsing

# Fetch models fom Google Storage

Next we can get pretrained checkpoint from Google Storage. For the sake of speed, this is a medium sized model trained on [TABFACT](https://tabfact.github.io/). Note that best results in the paper were obtained with a large model.

In [0]:
! gsutil cp "gs://tapas_models/2020_10_07/tapas_tabfact_inter_masklm_medium_reset.zip" "tapas_model.zip" && unzip tapas_model.zip
! mv tapas_tabfact_inter_masklm_medium_reset tapas_model

In [7]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())


[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 11180698882780347981
, name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 5765223632746339895
physical_device_desc: "device: XLA_CPU device"
, name: "/device:XLA_GPU:0"
device_type: "XLA_GPU"
memory_limit: 17179869184
locality {
}
incarnation: 952663202112676295
physical_device_desc: "device: XLA_GPU device"
]


# Imports

In [22]:
import tensorflow.compat.v1 as tf
import os 
import shutil
import csv
import pandas as pd
import IPython

tf.get_logger().setLevel('ERROR')

In [15]:
from tapas.utils import tf_example_utils
from tapas.protos import interaction_pb2
from tapas.utils import number_annotation_utils
import math
import json


# Load checkpoint for prediction

Here's the prediction code, which will create and `interaction_pb2.Interaction` protobuf object, which is the datastructure we use to store examples, and then call the prediction script.

In [5]:
os.makedirs('results/tabfact/tf_examples', exist_ok=True)
os.makedirs('results/tabfact/model', exist_ok=True)
with open('results/tabfact/model/checkpoint', 'w') as f:
  f.write('model_checkpoint_path: "model.ckpt-0"')
for suffix in ['.data-00000-of-00001', '.index', '.meta']:
  shutil.copyfile(f'tapas_model/model.ckpt{suffix}', f'results/tabfact/model/model.ckpt-0{suffix}')

In [6]:
max_seq_length = 512
vocab_file = "tapas_model/vocab.txt"
config = tf_example_utils.ClassifierConversionConfig(
    vocab_file=vocab_file,
    max_seq_length=max_seq_length,
    max_column_id=max_seq_length,
    max_row_id=max_seq_length,
    strip_column_names=False,
    add_aggregation_candidates=False,
)
converter = tf_example_utils.ToClassifierTensorflowExample(config)

def convert_interactions_to_examples(tables_and_queries):
  """Calls Tapas converter to convert interaction to example."""
  for idx, (table, queries) in enumerate(tables_and_queries):
    interaction = interaction_pb2.Interaction()
    for position, query in enumerate(queries):
      question = interaction.questions.add()
      question.original_text = query
      question.id = f"{idx}-0_{position}"
    for header in table[0]:
      interaction.table.columns.add().text = header
    for line in table[1:]:
      row = interaction.table.rows.add()
      for cell in line:
        row.cells.add().text = cell
    number_annotation_utils.add_numeric_values(interaction)
    for i in range(len(interaction.questions)):
      try:
        yield converter.convert(interaction, i)
      except ValueError as e:
        print(f"Can't convert interaction: {interaction.id} error: {e}")
        
def write_tf_example(filename, examples):
  with tf.io.TFRecordWriter(filename) as writer:
    for example in examples:
      writer.write(example.SerializeToString())

def predict(table_data, queries, verbose=False):
  table = [list(map(lambda s: s.strip(), row.split("#"))) 
         for row in table_data.strip().split("\n") if row.strip()]
  examples = convert_interactions_to_examples([(table, queries)])
  write_tf_example("results/tabfact/tf_examples/test.tfrecord", examples)
  write_tf_example("results/tabfact/tf_examples/dev.tfrecord", [])
  
  ! python -m tapas.run_task_main \
    --task="TABFACT" \
    --output_dir="results" \
    --noloop_predict \
    --test_batch_size={len(queries)} \
    --tapas_verbosity="ERROR" \
    --compression_type= \
    --reset_position_index_per_cell \
    --init_checkpoint="tapas_model/model.ckpt" \
    --bert_config_file="tapas_model/bert_config.json" \
    --mode="predict" 2> error


  results_path = "results/tabfact/model/test.tsv"
  all_results = []
  if verbose:
      df = pd.DataFrame(table[1:], columns=table[0])
      display(IPython.display.HTML(df.to_html(index=False)))
  
  with open(results_path) as csvfile:
      reader = csv.DictReader(csvfile, delimiter='\t')
      for row in reader:
          supported = int(row["pred_cls"])
          all_results.append(supported)
          if verbose:
              position = int(row['position'])
              if supported:
                  print("> SUPPORTS:", queries[position])
              else:
                  print("> REFUTES:", queries[position])
  return all_results

# Predict

In [16]:
# Based on TabFact table 2-1610384-4.html.csv
result = predict("""
tournament            | wins | top - 10 | top - 25 | events | cuts made
masters tournament    | 0    | 0        | 1        | 3      | 2        
us open               | 0    | 0        | 0        | 4      | 3        
the open championship | 0    | 0        | 0        | 2      | 1        
pga championship      | 0    | 1        | 1        | 4      | 2        
totals                | 0    | 1        | 2        | 13     | 8        
""", ["The most frequently occurring number of events is 4", "The most frequently occurring number of events is 3"])

is_built_with_cuda: True
is_gpu_available: False
GPUs: []
Training or predicting ...
Evaluation finished after training step 0.


In [7]:
dataset = json.load(open("E:\\TableFactChecking-LLMs\\data\\test_examples_with_csv_paraphrased.json"))

all_preds = []
all_labels = []

n=0
for key, value in dataset.items():
    if n == 10:
        break
    n+=1
    questions, labels, entity, table_csv = value
    
    # prediction
    preds = predict(table_csv, questions, verbose=True)

    all_preds.extend(preds)
    all_labels.extend(labels)


is_built_with_cuda: True
is_gpu_available: False
GPUs: []
Training or predicting ...
Evaluation finished after training step 0.


tournament,wins,top - 5,top - 10,top - 25,events,cuts made
masters tournament,0,1,2,4,4,4
us open,0,2,3,4,6,5
the open championship,1,2,2,2,3,3
pga championship,0,0,1,2,5,4
totals,1,5,8,12,18,16


> SUPPORTS: tony lema aims to secure a position within the top five at the master tournament, the us open, and the open championship
> SUPPORTS: The tournaments Tony Lema participated in consist of the Masters Tournament, the U.S. Open, the PGA Championship, and The Open Championship.
> SUPPORTS: Tony Lema's sole victory was at the Open Championship.
> SUPPORTS: tony lema fails to secure victory at the us open
> SUPPORTS: tony lema achieved a top 10 position in the pga championship yet fails to maintain his standing
> REFUTES: tony lema aims to secure a position within the top five at the pga championship, the us open, and the open championship.
> SUPPORTS: The tournaments Tony Lema did not compete in are the Masters Tournament, the U.S. Open, the PGA Championship, and the Open Championship.
> REFUTES: championship event victory achieved by tony lema was the pga championship
> REFUTES: tony lema fails to secure victory at the pga championship
> REFUTES: tony lema reached the us open to

player,span,start,tries,conv,pens,drop
vaea anitoni,1992 - 2000,44,26,0,0,0
paul emerick,2003 - 2012,49,17,0,0,0
todd clever,2003 -,51,11,0,0,0
philip eloff,2000 - 2007,34,10,0,0,0
takudzwa ngwenya,2007 -,27,10,0,0,0
chris wyles,2007 -,35,10,14,22,1
david fee,2002 - 2005,28,9,0,0,0
mike hercus,2002 - 2009,45,9,90,76,4
riaan van zyl,2003 - 2004,12,9,0,0,0


> REFUTES: paul emerick does not possess the highest count of attempts among the 9 players
> REFUTES: the highest frequency of drops experienced by a single player occurs throughout a duration of 8 years
> SUPPORTS: Three out of nine players have not completed their tenure in the United States national rugby union team.
> SUPPORTS: chris wyles was among the two players who made a drop during their tenure on the rugby team
> REFUTES: riann van zyl holds the record for the briefest tenure in the united states national rugby union squad and shares it with three others regarding the fewest try conversions.
> REFUTES: paul emerick's attempt count is not lower than that of every other participant
> REFUTES: the highest count of drops by a singular player occurs exclusively within a duration of one annum
> REFUTES: every player has completed their tenure with the United States national rugby union team.
> REFUTES: chris wyles was the sole player to experience a drop during his tenure on the r

...

> REFUTES: excluding slovakia, countries are located in north america
> SUPPORTS: There are five players hailing from both the United States and Canada.
> SUPPORTS: There are no instances where two players are associated with the same educational institution or youth sports organization.
> REFUTES: round 3 is the sole exception for list 2
> REFUTES: triston grant is the sole player designated for left wing position
> SUPPORTS: the sole player hailing from a nation other than the United States or Canada is Norwegian
> REFUTES: the count of players hailing from canada exceeds the count of players originating from the united states
> SUPPORTS: all players originate from a collegiate setup or a minor league/association team
> SUPPORTS: In rounds 3 and 9, there will be a draft involving two players.
> REFUTES: triston grant occupies the same role during the draft as another player selected in his round
is_built_with_cuda: True
is_gpu_available: False
GPUs: []
Training or predicting ...
Eval

year,beer name,prize,category,competition
1995,good old boy,overall winner,beer of the festival,camra white horse branch beer festival
1997,dr hexter 's healer,runner up,beer of the festival,camra northamptonshire branch beer festival
2002,good old boy,gold medal,best bitter,siba south east region beer competition
2002,good old boy,second reserve champion,overall,siba south east region beer competition
2002,maggs magnificent mild,gold medal,mild and porter,siba south east region beer competition
2002,mr chubb 's lunchtime bitter,silver medal,standard bitter,siba south east region beer competition
2002,dr hexter 's healer,silver medal,strong ale,siba south east region beer competition
2003,maggs magnificent mild,gold medal,mild and porter,siba national beer competition
2003,full circle,silver medal,best bitter,camra great british beer festival
2004,maggs magnificent mild,overall winner,beer of the festival,bracknell blues and real ale festival


> SUPPORTS: West Berkshire Brewery's Good Old Boy beer won 5 awards from 1995 to 2009.
> REFUTES: Dr. Hexter's Healer beer from West Berkshire Brewery earned 3 gold medals between 1995 and 2009.
> SUPPORTS: Between 1995 and 2009, Maggs' Magnificent Mild was West Berkshire Brewery's most decorated beer.
> SUPPORTS: Between 1995 and 2009, 3 beers from West Berkshire Brewery secured individual accolades.
> SUPPORTS: Between 1995 and 2009, West Berkshire Brewery's Maggs Magnificent Mild secured two awards at the SIBA National Beer Competition.
> REFUTES: Between 1995 and 2009, the Good Old Boy beer from West Berkshire Brewery failed to secure any awards.
> REFUTES: Between 1995 and 2009, Dr Hexter Healer beer from West Berkshire Brewery earned solely a silver medal.
> SUPPORTS: west berkshire brewery's maggs magnificent mild consistently secured a gold medal during its prize-winning years from 1995 to 2009
> REFUTES: Between 1995 and 2005, West Berkshire Brewery's Maggs Magnificent Mild se

game,date,team,location attendance,record
75,april 1,houston,us airways center 18422,41 - 34
76,april 3,sacramento,us airways center 18422,42 - 34
77,april 5,dallas,american airlines center 20301,42 - 35
78,april 8,new orleans,new orleans arena 17781,43 - 35
79,april 10,memphis,fedexforum 15908,43 - 36
80,april 11,minnesota,target center 18478,44 - 36
81,april 13,memphis,us airways center 18422,45 - 36
82,april 15,golden state,us airways center,46 - 36


> SUPPORTS: Three matches held at US Airways Center share identical crowd sizes.
> SUPPORTS: the greatest number of people present was at the american airline center
> SUPPORTS: the match scheduled at the target center occurs subsequent to the event held at the fedexforum
> SUPPORTS: the match with minnesota drew more spectators compared to the encounter against new orleans
> REFUTES: the game against new orleans did not feature the largest crowd
> REFUTES: attendance levels are identical for golden state, memphis, and new orleans
> REFUTES: the target center should have the largest number of attendees
> REFUTES: the match held in the fedexforum occurs subsequent to the event at the target center
> REFUTES: the match versus minnesota drew fewer spectators compared to the encounter against new orleans
> REFUTES: the match versus new orleans boasts the largest crowd


In [9]:
correct = sum([p==l for p,l in zip(all_preds, all_labels)])
total = len(all_labels)
accuracy = correct / total

print(f"Tapas accuracy for paraphrased dataset: {accuracy*100:.2f}% ({correct}/{total})")

Tapas accuracy for paraphrased dataset: 68.69% (68/99)


In [12]:
dataset_original = json.load(open("E:\\TableFactChecking-LLMs\\data\\test_examples_with_csv.json"))

all_preds_original = []
all_labels_original = []

n=0
for key, value in dataset_original.items():
    if n == 10:
        break
    n+=1
    questions, labels, entity, table_csv = value
    
    # prediction
    preds = predict(table_csv, questions, verbose=True)

    all_preds_original.extend(preds)
    all_labels_original.extend(labels)

is_built_with_cuda: True
is_gpu_available: False
GPUs: []
Training or predicting ...
Evaluation finished after training step 0.


tournament,wins,top - 5,top - 10,top - 25,events,cuts made
masters tournament,0,1,2,4,4,4
us open,0,2,3,4,6,5
the open championship,1,2,2,2,3,3
pga championship,0,0,1,2,5,4
totals,1,5,8,12,18,16


> SUPPORTS: tony lema be in the top 5 for the master tournament , the us open , and the open championship
> SUPPORTS: tournament that tony lema have participate in include the master tournament , the us open , the pga championship and the open championship
> SUPPORTS: the only tournament that tony lema win in be the open championship
> SUPPORTS: tony lema do not win in the us open
> REFUTES: tony lema make it to the top 10 in the pga championship , but do not continue on
> REFUTES: tony lema be in the top 5 for the pga championship , the us open , and the open championship
> REFUTES: tournament that tony lema have not participate in include the master tournament , the us open , the pga championship and the open championship
> REFUTES: tournament that tony lema won in be pga championship
> SUPPORTS: tony lema do not win in the pga championship
> REFUTES: tony lema make it to the top 10 in the us open , but do not continue on
is_built_with_cuda: True
is_gpu_available: False
GPUs: []
Trai

player,span,start,tries,conv,pens,drop
vaea anitoni,1992 - 2000,44,26,0,0,0
paul emerick,2003 - 2012,49,17,0,0,0
todd clever,2003 -,51,11,0,0,0
philip eloff,2000 - 2007,34,10,0,0,0
takudzwa ngwenya,2007 -,27,10,0,0,0
chris wyles,2007 -,35,10,14,22,1
david fee,2002 - 2005,28,9,0,0,0
mike hercus,2002 - 2009,45,9,90,76,4
riaan van zyl,2003 - 2004,12,9,0,0,0


> REFUTES: paul emerick do not have the greatest number of tries out of the 9 total player
> REFUTES: the greatest number of drop from 1 player happen over the span of 8 year
> SUPPORTS: 3 of the 9 player have not yet finish their span of play on the united state national rugby union team
> REFUTES: chris wyles be 1 of the 2 player with drop during their time on the rugby team
> REFUTES: riann van zyl have the shortest time span on the united state national rugby union team and tie with 3 others for the smallest number of tries
> REFUTES: paul emerick do not have fewer tries than any other player
> REFUTES: the greatest number of drop from 1 player happen over the span of only 1 year
> REFUTES: all of the player have already finish their span of play on the united state national rugby union team
> SUPPORTS: chris wyles be the only player with drop during his time on the rugby team
> REFUTES: riann van zyl have the longest time span on the united state national rugby union team and tie 

...

> REFUTES: except for slovakia , all other country be from north america
> REFUTES: 5 player be from the united state and canada each
> SUPPORTS: no 2 player attend the same college / junior / club team (league)
> SUPPORTS: only round 3 be not list 2 time
> SUPPORTS: triston grant be the only player list for left wing
> REFUTES: the only player not from the united state or canada be from norway
> SUPPORTS: the number of player from canada outnumber the number of player from the united state
> SUPPORTS: every player come from either a college program or a junior / club team
> REFUTES: 2 player be draft in each round round 3 and 9
> REFUTES: triston grant play the same position as the other payer draft in his round
is_built_with_cuda: True
is_gpu_available: False
GPUs: []
Training or predicting ...
Evaluation finished after training step 0.


year,beer name,prize,category,competition
1995,good old boy,overall winner,beer of the festival,camra white horse branch beer festival
1997,dr hexter 's healer,runner up,beer of the festival,camra northamptonshire branch beer festival
2002,good old boy,gold medal,best bitter,siba south east region beer competition
2002,good old boy,second reserve champion,overall,siba south east region beer competition
2002,maggs magnificent mild,gold medal,mild and porter,siba south east region beer competition
2002,mr chubb 's lunchtime bitter,silver medal,standard bitter,siba south east region beer competition
2002,dr hexter 's healer,silver medal,strong ale,siba south east region beer competition
2003,maggs magnificent mild,gold medal,mild and porter,siba national beer competition
2003,full circle,silver medal,best bitter,camra great british beer festival
2004,maggs magnificent mild,overall winner,beer of the festival,bracknell blues and real ale festival


> SUPPORTS: west berkshire brewery 's good old boy beer have 5 award between 1995 and 2009
> REFUTES: west berkshire brewery 's dr hexter healer beer win 3 gold medal between 1995 and 2009
> SUPPORTS: west berkshire brewery 's maggs magnificent mild be its most decorate beer between 1995 and 2009
> SUPPORTS: 3 of west berkshire brewery 's beer each win a single award between 1995 and 2009
> REFUTES: west berkshire brewery 's maggs magnificent mild 2 time win an award at the siba national beer competition between 1995 and 2009
> REFUTES: west berkshire brewery 's good old boy beer do not win a single award between 1995 and 2009
> REFUTES: west berkshire brewery 's dr hexter healer beer have only receive a silver medal between 1995 and 2009
> REFUTES: west berkshire brewery 's maggs magnificent mild have win a gold medal every single instance in which it win a prize between 1995 and 2009
> REFUTES: west berkshire brewery 's maggs magnificent mild 2 time receive a bronze award at the siba

game,date,team,location attendance,record
75,april 1,houston,us airways center 18422,41 - 34
76,april 3,sacramento,us airways center 18422,42 - 34
77,april 5,dallas,american airlines center 20301,42 - 35
78,april 8,new orleans,new orleans arena 17781,43 - 35
79,april 10,memphis,fedexforum 15908,43 - 36
80,april 11,minnesota,target center 18478,44 - 36
81,april 13,memphis,us airways center 18422,45 - 36
82,april 15,golden state,us airways center,46 - 36


> SUPPORTS: 3 of the game at the us airways center have the same attendance
> SUPPORTS: the highest attendance be at the american airline center
> SUPPORTS: the game at the target center take place after the game at the fedexforum
> SUPPORTS: the game against minnesota have a higher attendance than the game against new orleans
> REFUTES: the game against new orleans didn't have the highest attendance
> REFUTES: golden state , memphis , and new orleans all have the same attendance
> REFUTES: the highest attendance be at the target center
> REFUTES: the game at the fedexforum take place after the game at the target center
> REFUTES: the game against minnesota have a lower attendance than the game against new orleans
> REFUTES: the game against new orleans have the highest attendance


In [13]:
correct = sum([p==l for p,l in zip(all_preds_original, all_labels_original)])
total = len(all_labels_original)
accuracy = correct / total

print(f"Tapas accuracy for original dataset: {accuracy*100:.2f}% ({correct}/{total})")

Tapas accuracy for original dataset: 70.71% (70/99)
